<a href="https://colab.research.google.com/github/jzyee/water_stress_proj/blob/master/correlation_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this file is for checking the correlation of the type of climate data available

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
% cd /content/gdrive/'My Drive'/water_stress_detection

/content/gdrive/My Drive/water_stress_detection


In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.pylabtools import figsize
figsize(14, 6)
import statsmodels as sm
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.arima_process import ArmaProcess
#from causalimpact import CausalImpact
import datetime as dt
from matplotlib import pyplot
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
#from statsmodels.tsa.seasonal import STL
import plotly.tools as tls
import math
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# E-OBS

In [4]:
! ls data_files/monthly_means

monthly_mean_pp_EBTRN.csv  monthly_mean_tg_EBTRN.csv  monthly_mean_tx_EBTRN.csv
monthly_mean_rr_EBTRN.csv  monthly_mean_tn_EBTRN.csv


In [ ]:
climate_data_path = 'data_files/monthly_means/'


def combine_eob(site_id):
  type_list = ['pp', 'rr', 'tg', 'tn', 'tx']

  folder_path = 'data_files/monthly_means/'

  # file naming convention
  # file_cov = 'wind_monthly_'
  # file_name = 'SWIT345'

  start_name_cov = 'monthly_mean_'
  end_name_cov = '_EBTRN'

  out_df = None

  for idx, data_type in enumerate(type_list):
    file_path = folder_path + start_name_cov + data_type + end_name_cov + '.csv'
    if idx == 0:
      df = pd.read_csv(file_path, parse_dates=True).set_index(['Date'])[[site_id]]
      df = df.rename(columns={site_id: data_type})
    
    else:
      df[data_type] = pd.read_csv(file_path, parse_dates=True).set_index(['Date'])[site_id]

  
  return df

def climate_ndvi_corr(site_id):
  harm_df = pd.read_csv('data_files/processed_harm_ndvi.csv', parse_dates=True)
  harm_df.index = pd.to_datetime(harm_df.iloc[:,0], format='%Y-%m-%d') 
  harm_df = harm_df.iloc[:,1:]
  harm_df.index.name = 'Date'
  # filter out the useable dates
  harm_df = harm_df.loc[(harm_df.index > '1986') & (harm_df.index < '2019')]
  # get the climate data
  df = combine_eob(site_id)
  df = df.loc[(df.index > '1986') & (df.index < '2019')]

  df['ndvi'] = harm_df[site_id]
  return df.corr()[['ndvi']]

site_id_list = ['SWIT345', 'E45']


for site_id in site_id_list:
  harm_df = pd.read_csv('data_files/processed_harm_ndvi.csv', parse_dates=True)
  harm_df.index = pd.to_datetime(harm_df.iloc[:,0], format='%Y-%m-%d') 
  harm_df = harm_df.iloc[:,1:]
  harm_df.index.name = 'Date'
  # filter out the useable dates
  harm_df = harm_df.loc[(harm_df.index > '1986') & (harm_df.index < '2019')]
  # get the climate data
  df = combine_eob(site_id)
  df = df.loc[(df.index > '1986') & (df.index < '2019')]

  df['ndvi'] = harm_df[site_id]
  display(df.corr())




,pp,rr,tg,tn,tx,ndvi
pp,1.000000,-0.443312,-0.346878,-0.349077,-0.332437,-0.275084
rr,-0.443312,1.000000,0.300026,0.330259,0.266517,0.319883
tg,-0.346878,0.300026,1.000000,0.996641,0.996979,0.836390
tn,-0.349077,0.330259,0.996641,1.000000,0.987969,0.848305
tx,-0.332437,0.266517,0.996979,0.987969,1.000000,0.822677
ndvi,-0.275084,0.319883,0.836390,0.848305,0.822677,1.000000


,pp,rr,tg,tn,tx,ndvi
pp,1.000000,-0.286402,-0.285329,-0.309762,-0.261922,-0.107064
rr,-0.286402,1.000000,-0.241138,-0.200541,-0.272062,-0.027290
tg,-0.285329,-0.241138,1.000000,0.994028,0.996175,0.383863
tn,-0.309762,-0.200541,0.994028,1.000000,0.980807,0.368943
tx,-0.261922,-0.272062,0.996175,0.980807,1.000000,0.395342
ndvi,-0.107064,-0.027290,0.383863,0.368943,0.395342,1.000000


In [ ]:
corr_df = None

for idx, site_id in enumerate(harm_df):
  if idx == 0:
    df = climate_ndvi_corr(site_id)
    if not df.isnull().values.any():
      corr_df = df.rename(columns={'ndvi': site_id})
  else:
    df = climate_ndvi_corr(site_id)
    if not df.isnull().values.any():
      corr_df[site_id] = df
    



corr_mean_df = corr_df.mean(axis=1)

# UK67 has no values
display(corr_df)
display(corr_mean_df)

,FMMOL,E12,ELFI,BBOB,VERMTL,CAN,befa,TZEI,GERM210,FELSM,CRO,OGEW,FSGW,FOU,SWIT259,VERM11,UK99,OSDS,Kom,SERR15,WESC,TOC,BOS_PER,OeS,OR2,Sinca3,OSSA06,CRO_CUD,MBA,E17,UK62,SZBH,CLFS,Sinca6,KNJB,E07,DRFS,QPOM,Gks,UnS,...,SHTS,CRO_RAM,RIED,DOFS,MSC,LR3,M04,MWSB,E09,PENNA,CEB,E39,UK10,FPTRE2,FSSW,QGHC,GERM206,DK1014,ROM_SEB,KAMP,M27,mos,SLO_SUT,HSKM,MON1,PIOV,NGMW,PWTS,FS30,s121,BALD,Runcu2,Z1,ITAL037,AGNE,LE50,NGLS,AFO,M11,BB
pp,-0.224869,0.034679,-0.332238,-0.147710,-0.479920,-0.390938,-0.399217,-0.265491,-0.050134,-0.178310,-0.306624,-0.261420,-0.001974,-0.204533,-0.318468,-0.510566,0.048699,-0.262010,-0.348190,-0.458663,-0.128364,-0.287383,-0.200973,-0.036493,-0.208384,-0.502530,-0.519951,-0.256448,-0.338699,-0.304678,0.092506,-0.212417,-0.496289,-0.435103,-0.228522,-0.228565,-0.417256,-0.132068,-0.373462,-0.051303,...,-0.012972,-0.249557,-0.197068,-0.381970,-0.308341,-0.421512,-0.414784,-0.204337,-0.282633,-0.322998,-0.403811,-0.199937,0.031120,-0.252797,-0.032799,-0.329998,-0.198543,0.106492,-0.381165,0.056438,-0.480337,-0.226772,-0.286652,-0.163838,-0.091289,-0.348616,-0.229611,-0.032331,-0.255029,-0.155821,0.002176,-0.445167,-0.371251,-0.272079,-0.282670,-0.298776,-0.339354,0.046352,-0.503661,-0.007700
rr,-0.114202,-0.004730,-0.005156,-0.026452,-0.196349,0.308213,0.432683,0.479523,0.045618,0.308671,0.252403,0.423728,0.132561,-0.006770,0.291536,-0.221134,-0.207067,-0.302095,0.236619,-0.070104,-0.012746,0.116499,-0.059875,0.026209,-0.204515,0.538417,-0.209953,-0.079402,0.284681,-0.172974,-0.134825,0.130074,0.131830,0.517431,0.182886,0.045822,0.463935,-0.090921,0.523935,0.033287,...,0.243840,-0.170136,0.219216,0.372429,-0.020881,-0.188481,-0.033557,0.351728,0.051419,-0.282758,0.381562,0.002518,-0.032805,0.146812,0.209349,0.184425,-0.032795,-0.046905,0.340210,0.118216,-0.051750,0.153825,0.214940,0.008224,0.116430,0.244268,0.249164,-0.013948,-0.096665,0.077446,0.151129,0.352216,0.474048,0.070008,-0.143966,-0.230909,0.294210,-0.099109,-0.041350,0.256310
tg,0.693024,0.507679,0.817982,0.710083,0.781243,0.825824,0.797353,0.828718,0.490483,0.638264,0.761963,0.714055,0.739287,0.618351,0.875437,0.836020,0.684824,0.712582,0.481178,0.753271,0.348589,0.697900,0.663121,0.401529,0.726040,0.890025,0.780860,0.771177,0.839522,0.782872,0.492595,0.725019,0.778333,0.859184,0.654079,0.842493,0.826019,0.760993,0.811553,0.713929,...,0.729688,0.792069,0.702216,0.755581,0.544580,0.859386,0.638790,0.572284,0.831887,0.837779,0.798080,0.826945,0.579438,0.817596,0.774174,0.711931,0.648359,0.274106,0.676405,0.756062,0.817328,0.662714,0.775423,0.745341,0.771473,0.774104,0.704660,0.011264,0.656004,0.572119,0.781836,0.850403,0.711013,0.357321,0.687451,0.751702,0.811543,0.634079,0.844409,0.718958
tn,0.668603,0.491367,0.806903,0.720738,0.779644,0.822691,0.798365,0.821522,0.497675,0.646698,0.767506,0.711766,0.722098,0.609982,0.866753,0.838565,0.661914,0.704917,0.470749,0.753470,0.348092,0.702099,0.656913,0.422369,0.732040,0.886392,0.777071,0.762425,0.838308,0.790039,0.492680,0.726240,0.782231,0.856253,0.646734,0.848264,0.826594,0.760683,0.821131,0.726311,...,0.712198,0.784650,0.718857,0.745899,0.562629,0.854606,0.650160,0.574508,0.843898,0.814781,0.804785,0.826599,0.556211,0.820739,0.783207,0.694810,0.664226,0.276137,0.665669,0.755815,0.805436,0.678274,0.771119,0.747662,0.777888,0.775671,0.704425,0.017904,0.656757,0.565880,0.788515,0.852140,0.712940,0.377217,0.675920,0.753523,0.809988,0.643164,0.841942,0.724535
tx,0.705383,0.523553,0.810650,0.693353,0.780619,0.827097,0.791016,0.825949,0.467408,0.629762,0.750659,0.712855,0.735487,0.599386,0.876705,0.827885,0.694527,0.716578,0.484487,0.746895,0.344194,0.690378,0.655935,0.384259,0.710587,0.885686,0.777022,0.767639,0.834187,0.770635,0.495023,0.711073,0.766488,0.848926,0.647083,0.827743,0.817354,0.752073,0.792595,0.691745,...,0.717884,0.794461,0.681233,0.753242,0.531787,0.852818,0.626236,0.564807,0.816508,0.844111,0.786882,0.813516,0.584135,0.810411,0.752566,0.710516,0.616253,0.269015,0.680316,0.734128,0.816106,0.638

pp     -0.222893
rr      0.121058
tg      0.674579
tn      0.676189
tx      0.664390
ndvi    1.000000
dtype: float64

# writing the correlation dataframe to csv

In [ ]:
corr_df.to_csv('./correlation_data/corr_df.csv')

In [ ]:
corr_mean_df.to_csv('./correlation_data/corr_mean_df.csv')